# module03_embeddings_search


## embeddings.py
embeddings.py
Explanation of Embeddings
Embeddings are dense vector representations of text, images, or other data.
They capture semantic meaning and relationships between items.
Common types: Word embeddings, sentence embeddings, image embeddings.


In [ ]:
try:  # Try block to handle missing dependencies gracefully
    from sentence_transformers import SentenceTransformer  # Import sentence transformer for text embeddings
    import numpy as np  # Import NumPy for array operations

except ImportError:  # Handle case where sentence-transformers is not installed
    print("sentence-transformers not installed. Install with: pip install sentence-transformers")  # Print installation instruction
    print("Embeddings are vector representations that capture semantic meaning.")  # Explain what embeddings are
    print("Example: 'king' - 'man' + 'woman' ≈ 'queen' in word embeddings.")  # Give famous word embedding example

In [ ]:
# Load a sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Load pre-trained sentence transformer model


In [ ]:
def get_embeddings(texts):  # Define function to generate embeddings for texts
        """
        Generate embeddings for a list of texts.
        """
        return model.encode(texts)  # Use the model to encode texts into embeddings


In [ ]:
# Example texts
texts = [  # Define sample texts for embedding generation
        "The cat sat on the mat",  # First text sample
        "A feline rested on the rug",  # Second text sample (similar meaning to first)
        "The dog played in the park",  # Third text sample (different topic)
        "Machine learning is fascinating"  # Fourth text sample (different domain)
]


In [ ]:
# Generate embeddings
embeddings = get_embeddings(texts)  # Generate embeddings for all texts


In [ ]:
print("Embeddings shape:", embeddings.shape)  # Print shape of embeddings array (n_texts, embedding_dim)
print("First embedding (first 10 values):", embeddings[0][:10])  # Print first 10 values of first embedding

In [ ]:
# Calculate similarities
from sklearn.metrics.pairwise import cosine_similarity  # Import cosine similarity function
similarities = cosine_similarity(embeddings)  # Calculate pairwise similarities between all embeddings

In [ ]:
print("\nSimilarity matrix:")  # Print header for similarity matrix
for i, text in enumerate(texts):  # Iterate through texts with indices        print(f"{i}: {text}")  # Print index and corresponding text
    print(f"{i}: {text}")  # Print index and corresponding text
print(similarities)  # Print the full similarity matrix

Embedding models:
- Computer Vision: ResNet, VGG, Vision Transformer (ViT)
- NLP: Word2Vec, GloVe, BERT, Sentence Transformers
- Audio: Wav2Vec, HuBERT


Sentence Transformers are specifically designed for sentence-level embeddings
all-MiniLM-L6-v2 is a popular model for semantic similarity tasks


## vector_keyword_search.py
vector_keyword_search.py
Explanation of Vector Keyword Search
Vector keyword search uses techniques like TF-IDF and BM25 to rank documents based on keyword relevance.
TF-IDF: Term Frequency-Inverse Document Frequency - measures importance of terms in documents
BM25: Best Matching 25, an improved ranking function that considers term frequency and document length
Import required libraries for text processing and similarity calculations
TfidfVectorizer converts text documents to TF-IDF feature vectors


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# cosine_similarity calculates similarity between vectors using cosine distance
from sklearn.metrics.pairwise import cosine_similarity
# numpy for numerical operations and array handling
import numpy as np


In [ ]:
# Sample documents to demonstrate keyword-based search
# These documents contain overlapping keywords for testing relevance ranking
documents = [
    "The cat sat on the mat",        # Contains "cat"
    "The dog played in the park",    # Contains "dog"
    "Cats and dogs are pets",        # Contains both "cat" and "dog"
    "The weather is nice today"      # Contains neither keyword
]


In [ ]:
# Create TF-IDF vectorizer to convert text to numerical vectors
# TF-IDF weighs terms by importance: high in specific doc, low across all docs
vectorizer = TfidfVectorizer()
# Fit the vectorizer to documents and transform them into TF-IDF matrix
# Each row is a document, each column is a term, values are TF-IDF scores
tfidf_matrix = vectorizer.fit_transform(documents)


In [ ]:
# Define TF-IDF search function
def tfidf_search(query, documents, vectorizer, tfidf_matrix):
    """
    Perform TF-IDF based search.
    Converts query to TF-IDF vector and finds most similar documents.
    """
    # Transform the query into TF-IDF vector using the same vectorizer
    # This ensures query and documents use the same vocabulary/features
    query_vector = vectorizer.transform([query])

    # Calculate cosine similarity between query vector and all document vectors
    # cosine_similarity returns a matrix; flatten() converts to 1D array
    similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()

    # Sort indices in descending order (highest similarity first)
    # np.argsort gives indices in ascending order, [::-1] reverses it
    ranked_indices = np.argsort(similarities)[::-1]

    # Return list of (document, similarity_score) tuples in ranked order
    return [(documents[i], similarities[i]) for i in ranked_indices]

In [ ]:
# BM25 implementation (simplified version for educational purposes)
class SimpleBM25:
    # Initialize BM25 model with documents and parameters
    def __init__(self, documents):
        # Store the original documents for scoring
        self.documents = documents

        # Dictionary to store document frequency for each term
        # Document frequency = number of documents containing the term
        self.doc_freq = {}

        # Calculate document lengths (number of words in each document)
        self.doc_lengths = [len(doc.split()) for doc in documents]

        # Calculate average document length across all documents
        self.avg_doc_length = np.mean(self.doc_lengths)

        # BM25 parameters: k1 controls term frequency scaling, b controls length normalization
        self.k1 = 1.5  # BM25 parameter for term frequency saturation
        self.b = 0.75  # BM25 parameter for document length normalization

        # Calculate document frequencies for all unique terms
        for doc in documents:
            # Convert to lowercase and split into words, use set to get unique terms
            words = set(doc.lower().split())
            # Count how many documents contain each term
            for word in words:
                self.doc_freq[word] = self.doc_freq.get(word, 0) + 1

    # Calculate BM25 scores for a query against all documents
    def score(self, query):
        # Split query into individual terms
        query_terms = query.lower().split()

        # List to store BM25 scores for each document
        scores = []

        # Calculate score for each document
        for doc_idx, doc in enumerate(self.documents):
            # Initialize score for this document
            score = 0

            # Get document length (number of words)
            doc_length = self.doc_lengths[doc_idx]

            # Convert document to lowercase words for term matching
            doc_words = doc.lower().split()

            # Calculate score contribution from each query term
            for term in query_terms:
                # Only score if the term appears in this document
                if term in doc_words:
                    # Term frequency: how many times term appears in this document
                    tf = doc_words.count(term)

                    # Document frequency: how many documents contain this term
                    df = self.doc_freq.get(term, 0)

                    # Inverse document frequency: rarer terms get higher weight
                    # Uses Laplace smoothing (+0.5) to avoid division by zero
                    idf = np.log((len(self.documents) - df + 0.5) / (df + 0.5))

                    # BM25 scoring formula: combines TF, IDF, and length normalization
                    # (tf * (k1 + 1)) / (tf + k1 * (1 - b + b * doc_length/avg_length))
                    score += idf * (tf * (self.k1 + 1)) / (tf + self.k1 * (1 - self.b + self.b * doc_length / self.avg_doc_length))

            # Add this document's score to the results list
            scores.append(score)

        # Return BM25 scores for all documents
        return scores

In [ ]:
# Create BM25 model instance with our documents
bm25 = SimpleBM25(documents)


In [ ]:
# Define BM25 search function
def bm25_search(query, bm25_model, documents):
    """
    Perform BM25 search.
    Uses BM25 scoring to rank documents by relevance to query terms.
    """
    # Get BM25 scores for all documents
    scores = bm25_model.score(query)

    # Sort document indices by BM25 score in descending order
    ranked_indices = np.argsort(scores)[::-1]

    # Return list of (document, score) tuples in ranked order
    return [(documents[i], scores[i]) for i in ranked_indices]

In [ ]:
# Example usage demonstrating both TF-IDF and BM25 search
query = "cat dog"


In [ ]:
# Perform TF-IDF search and display results
print("TF-IDF Search Results:")
tfidf_results = tfidf_search(query, documents, vectorizer, tfidf_matrix)
# Show top 3 results with formatted scores
for doc, score in tfidf_results[:3]:
    print(f"Score: {score:.3f} - {doc}")


In [ ]:
# Perform BM25 search and display results
print("\nBM25 Search Results:")
bm25_results = bm25_search(query, bm25, documents)
# Show top 3 results with formatted scores
for doc, score in bm25_results[:3]:
    print(f"Score: {score:.3f} - {doc}")


## semantic_search.py
semantic_search.py
Explanation of Semantic Search
Semantic search goes beyond keyword matching to understand the meaning of words and context.
It uses embeddings to represent text semantically, allowing for more accurate retrieval.
Simple example using sentence transformers for semantic similarity
Try to import required libraries for semantic search functionality


In [ ]:
try:
    # Import SentenceTransformer for generating text embeddings
    from sentence_transformers import SentenceTransformer
    # Import cosine_similarity for measuring similarity between embeddings
    from sklearn.metrics.pairwise import cosine_similarity
    # Import numpy for numerical operations and array handling
    import numpy as np

except ImportError:
    # Print helpful error message with installation instructions
    print("sentence-transformers not installed. Install with: pip install sentence-transformers")

# Additional explanation of semantic search concepts
print("Semantic search uses embeddings to understand meaning rather than exact keywords.")
print("Example: Query 'feline resting' would match 'cat sleeping' due to semantic similarity.")

In [ ]:
# Load a pre-trained sentence transformer model for embedding generation
# 'all-MiniLM-L6-v2' is a lightweight model that converts text to 384-dimensional vectors
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
# Define the main semantic search function
def semantic_search(query, documents, model, top_k=3):
        """
        Perform semantic search using embeddings.
        This function converts both query and documents to embeddings,
        then finds the most semantically similar documents to the query.
        """
        # Encode the query text into a single embedding vector
        # model.encode([query]) returns a list, so we take the first (and only) element [0]
        query_embedding = model.encode([query])[0]

        # Encode all documents into embedding vectors
        # This creates a matrix where each row is a document's embedding
        doc_embeddings = model.encode(documents)

        # Calculate cosine similarity between query embedding and all document embeddings
        # cosine_similarity returns a matrix, so we take the first row [0] for our single query
        similarities = cosine_similarity([query_embedding], doc_embeddings)[0]

        # Get the indices of the top-k most similar documents
        # np.argsort sorts in ascending order, [::-1] reverses to descending, [:top_k] takes top k
        top_indices = np.argsort(similarities)[::-1][:top_k]

        # Create results list with document text and similarity scores
        # Pair each top document with its similarity score
        results = [(documents[i], similarities[i]) for i in top_indices]

        # Return the top-k most semantically similar documents with their scores
        return results

In [ ]:
# Example documents to demonstrate semantic search
# These documents contain both relevant and irrelevant content
documents = [
        "The cat is sleeping on the couch",        # Directly related to query
        "A feline rests on furniture",            # Semantically similar (feline = cat)
        "Dogs love to play fetch",                # Related to pets but different animal
        "The weather is beautiful today",         # Unrelated topic
        "Pets need regular veterinary care"       # Related to pets in general
]

In [ ]:
# Example query that should match semantically similar documents
query = "My cat is resting"

In [ ]:
# Perform semantic search and get top 3 results
results = semantic_search(query, documents, model)

In [ ]:
# Display the search results
print(f"Semantic search results for '{query}':")
# Print each result with its similarity score formatted to 3 decimal places
for doc, score in results:
        print(f"Score: {score:.3f} - {doc}")
